# yield from 的基本作用

## 一、yield from 会预激生成器

In [58]:
def gen_1():    # send什么就产生什么（3次）
    x = None
    for i in range(3):
        x = yield x
    yield x

g = gen_1()
next(g)     # 预激

In [59]:
g.send(3)

3

In [60]:
g.send(5)

5

In [61]:
g.send(7)

7

In [63]:
def ym_1():
    yield from gen_1()

g = ym_1()
next(g)   # 这一步是激活ym_1本身，而对gen_1无需激活

In [64]:
g.send(3)

3

In [65]:
g.send(5)

5

In [66]:
g.send(7)

7

## 二、yield from 捕获子生成器StopIteration时的return值

* for循环 不会显式的发出StopIteration异常，而是自动捕获StopIteration异常，从而无法捕获生成器的return值

In [67]:
def gen_2():
    for i in range(5):
        if i == 3:
            return 'exit'
        else:
            yield i

for i in gen_2():
    print(i)   

0
1
2


* 利用next产生的StopIteration捕获return值

In [70]:
g = gen_2()
try:
    while True:
        print(next(g))
except StopIteration as si:    # return的值会在StopIteration的value属性记录，利用next以及try-except捕获StopIteration从而获得return值
    print(si.value)

0
1
2
exit


* 利用 yield from捕获return值

In [73]:
def ym_2():
    catch = yield from gen_2()     # “继承”原始生成器的数据生成方式，同时在发生StopIteration时将原return的值传给catch
    print("副作用,捕获子生成器的return结果：", catch)                   # 打印捕获的return

for i in ym_2():
    print("打印数据", i)       # 打印ym_2()生成的每个数据

打印数据 0
打印数据 1
打印数据 2
副作用,捕获子生成器的return结果： exit


# 一个例子

In [43]:
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

* ### 基于循环

In [44]:
for k, v in data.items():
    sex, unit = k.split(';')
    cnt = len(v)
    avg = sum(v) / cnt
    print('{:2} {:5} averaging {:.2f}{}'.format(cnt, sex, avg, unit))

10 girls averaging 42.04kg
10 girls averaging 1.43m
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m


* ### 基于 yield from,调用方——委派生成器——子生成器模式

1.最初想法（需要处理异常）

In [52]:
from collections import namedtuple

line_res = namedtuple('Line', 'cnt avg')  

# 假想一个“流输入”，内部计数器与累加器不断计算，直至接受到None，则停止计算，返回计数及均值(子生成器)
def calc():
    sum_ = 0
    cnt = 0
    while True:
        value = yield 
        if value is None:
            avg = sum_ / cnt if cnt != 0 else None
            return line_res(cnt, avg)
        else:
            sum_ += value
            cnt += 1

            
# 将每一次结果存入res_dict(委派生成器)
def designate(res_list):
    line_res = yield from calc()     # line_res 捕获运算结果
    res_list.append(line_res)

    
def execute(data):
    res_list = []
    for k, v in data.items():
        sex, unit = k.split(';')
        des = designate(res_list)
        next(des)  # 启动委派生成器
        for value in v:
            # 不用预激子生成器了
            des.send(value)
        try:
            des.send(None)
        except StopIteration:    # 由于发送None导致委派生成器本身也终止，捕获该异常,使程序不会中断
            pass
        finally:    
            line_res = res_list[-1]
            print('{:2} {:5} averaging {:.2f}{}'.format(line_res.cnt, sex, line_res.avg, unit))

            
execute(data)   

10 girls averaging 42.04kg
10 girls averaging 1.43m
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m


2.改进（不用处理异常，实时打印）

In [54]:
import queue


def calc():
    sum_ = 0
    cnt = 0
    while True:
        value = yield 
        if value is None:
            avg = sum_ / cnt if cnt != 0 else None
            return cnt, avg
        else:
            sum_ += value
            cnt += 1

def designate(res_id, q):
    cnt, avg = yield from calc()
    sex, unit = res_id.split(';')
    resline = (cnt, sex, avg, unit)
    q.put(resline)
    yield           # 避免出现StopIteration

    
def execute(data):
    q = queue.Queue(maxsize=1)      # 队列中每次最多一个结果
    for k, v in data.items():
        des = designate(k, q)
        next(des) 
        for value in v:
            des.send(value)
        des.send(None)
        resline = q.get()
        print('{:2} {:5} averaging {:.2f}{}'.format(*resline))

        
execute(data)

10 girls averaging 42.04kg
10 girls averaging 1.43m
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m


## 三、离散仿真例子（出租车）

In [190]:
import random

class Event(namedtuple('RawEvent', 'time proc action')):
    """ 作为有序队列的基本单元使用 """
    def __lt__(self,other): 
        return self.time < other.time


def trip():
    proc, time = yield           # 接收车辆 及 开始时间
    yield Event(time, proc, 'leave garge')
    cnt = random.randint(1,5)    # 随机载1-5趟乘客
    for i in range(cnt):
        for evt in ['pick up passenger', 'drop off passenger']:
            time += random.randint(1,10)
            yield Event(time, proc, evt)
    time += random.randint(1,10)  
    yield Event(time, proc, 'going home')
       
        
def depart(n_cars, interval_time, threshold):
    """ 发车数量， 发车间隔时间， 最大时间"""
    q = queue.PriorityQueue()   # 事件记录
    time = 0
    cnt = 0    # 还未回家的车
    for i in range(n_cars):
        t = trip()
        next(t)
        start = t.send((i, time))
        q.put(start)
        for evt in t:
            if evt.time > threshold:
                cnt += 1
                break
            else:
                q.put(evt)
        time += interval_time
    
    while not q.empty():
        evt = q.get()
        taxi_id = evt.proc
        print('taxi: {} {}{}'.format(taxi_id, ' '*(3*taxi_id), evt))
    if cnt == 0:
        print("end of events")
    else:
        print("end of simulation time: %d cars still go on" % cnt)

In [191]:
depart(3,5,100)

taxi: 0 Event(time=0, proc=0, action='leave garge')
taxi: 1    Event(time=5, proc=1, action='leave garge')
taxi: 0 Event(time=6, proc=0, action='pick up passenger')
taxi: 0 Event(time=7, proc=0, action='drop off passenger')
taxi: 2       Event(time=10, proc=2, action='leave garge')
taxi: 1    Event(time=15, proc=1, action='pick up passenger')
taxi: 0 Event(time=15, proc=0, action='going home')
taxi: 2       Event(time=19, proc=2, action='pick up passenger')
taxi: 2       Event(time=21, proc=2, action='drop off passenger')
taxi: 1    Event(time=23, proc=1, action='drop off passenger')
taxi: 2       Event(time=30, proc=2, action='pick up passenger')
taxi: 1    Event(time=32, proc=1, action='pick up passenger')
taxi: 2       Event(time=34, proc=2, action='drop off passenger')
taxi: 1    Event(time=37, proc=1, action='drop off passenger')
taxi: 2       Event(time=37, proc=2, action='pick up passenger')
taxi: 1    Event(time=42, proc=1, action='going home')
taxi: 2       Event(time=46, proc

In [192]:
depart(3,5,30)

taxi: 0 Event(time=0, proc=0, action='leave garge')
taxi: 0 Event(time=3, proc=0, action='pick up passenger')
taxi: 1    Event(time=5, proc=1, action='leave garge')
taxi: 0 Event(time=10, proc=0, action='drop off passenger')
taxi: 2       Event(time=10, proc=2, action='leave garge')
taxi: 2       Event(time=11, proc=2, action='pick up passenger')
taxi: 2       Event(time=12, proc=2, action='drop off passenger')
taxi: 1    Event(time=15, proc=1, action='pick up passenger')
taxi: 1    Event(time=16, proc=1, action='drop off passenger')
taxi: 2       Event(time=17, proc=2, action='pick up passenger')
taxi: 0 Event(time=18, proc=0, action='pick up passenger')
taxi: 0 Event(time=20, proc=0, action='drop off passenger')
taxi: 1    Event(time=20, proc=1, action='pick up passenger')
taxi: 2       Event(time=25, proc=2, action='drop off passenger')
taxi: 1    Event(time=25, proc=1, action='drop off passenger')
taxi: 1    Event(time=26, proc=1, action='pick up passenger')
taxi: 0 Event(time=27, 